## Importing Library

In [2]:
# prediction
def check_correct(predict, y):
    result = {}
    result['cancer-correct'] = 0
    result['cancer-wrong'] = 0
    result['normal-correct'] = 0
    result['normal-wrong'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['normal-correct'] += 1
            else :
                result['cancer-correct'] += 1
        else :
            if y[i] == 0 :
                result['normal-wrong'] += 1
            else :
                result['cancer-wrong'] += 1

    #for result_k, result_v in result.items():
        #print(result_k +" : "+ str(result_v))
    sensitivity=result['cancer-correct']/(result['cancer-correct']+result['cancer-wrong'])
    specificity=result['normal-correct']/(result['normal-correct']+result['normal-wrong'])
    #print("Sensitivity :", sensitivity)
    #print("Specificity :", specificity)
    return sensitivity, specificity

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=15)

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

num=1

print(tf.__version__)

1.10.0


## Import Data

In [3]:
path = "C://test/TCGA_with_GEO/"
types = ["TCGA_with_GEO_input_ensemble_CV_3000", "TCGA_with_GEO_input_ensemble_foundation_308",
         "TCGA_with_GEO_input_ensemble_foundation_2267", "TCGA_with_GEO_input_ensemble_Mean_3000",
         "TCGA_with_GEO_input_ensemble_VAR_3000"
         ]
select = [4, 0, 1]
idx_col = 0

In [4]:
file_1 = path+types[select[0]]+".csv"
data_1 = pd.read_csv(file_1,index_col=idx_col)
sample_1,features_1 = data_1.shape
features_1 = features_1-3
print("[1] file_name: ", types[select[0]], "\nsample : {}  \nfeatures : {}".format(sample_1,features_1))
data_1.iloc[0:5, 0:5]

[1] file_name:  TCGA_with_GEO_input_ensemble_VAR_3000 
sample : 26248  
features : 3000


,XIST,CMPK2,MMP1,EGFR,RPS4Y1
patient,,,,,
GSM1000612,1.390,-1.111,-0.231,-0.107,0.072
GSM1000613,-1.500,0.131,-0.145,-2.664,-0.053
GSM1000614,0.485,1.006,-0.259,-0.039,0.163
GSM1000615,0.702,-0.035,0.298,1.075,0.101
GSM1000616,-0.375,-0.094,0.192,0.339,-0.009


In [5]:
file_2 = path+types[select[1]]+".csv"
data_2 = pd.read_csv(file_2,index_col=idx_col)
sample_2,features_2 = data_2.shape
features_2 = features_2-3
print("[2] file_name: ", types[select[1]], "\nsample : {}  \nfeatures : {}".format(sample_2,features_2))
data_2.iloc[0:5, 0:5]

[2] file_name:  TCGA_with_GEO_input_ensemble_CV_3000 
sample : 26248  
features : 3000


,DEPDC7,SGK2,RHEBL1,PLK4,IGKC
patient,,,,,
GSM1000612,0.442,-1.856,0.096,0.666,0.343
GSM1000613,0.381,-0.129,-0.808,-1.498,0.624
GSM1000614,-0.271,-0.719,0.216,-0.363,-1.160
GSM1000615,-0.525,0.048,0.127,-0.556,0.644
GSM1000616,0.285,0.773,1.061,0.769,-0.270


In [6]:
file_3 = path+types[select[2]]+".csv"
data_3 = pd.read_csv(file_3,index_col=idx_col)
sample_3,features_3 = data_3.shape
features_3 = features_3-3
print("[3] file_name: ", types[select[2]], "\nsample : {}  \nfeatures : {}".format(sample_3,features_3))
data_1.iloc[0:5, 0:5]

[3] file_name:  TCGA_with_GEO_input_ensemble_foundation_308 
sample : 26248  
features : 308


,XIST,CMPK2,MMP1,EGFR,RPS4Y1
patient,,,,,
GSM1000612,1.390,-1.111,-0.231,-0.107,0.072
GSM1000613,-1.500,0.131,-0.145,-2.664,-0.053
GSM1000614,0.485,1.006,-0.259,-0.039,0.163
GSM1000615,0.702,-0.035,0.298,1.075,0.101
GSM1000616,-0.375,-0.094,0.192,0.339,-0.009


## Split Train Test Data

In [7]:
train_p_ids = list(pd.read_csv(path+"Train_14_index.csv").x)
test_p_ids = list(pd.read_csv(path+"Test_14_index.csv").x)

In [8]:
train_data_1 = data_1.iloc[train_p_ids]
test_data_1 = data_1.iloc[test_p_ids]
y_val_1 = train_data_1.result
x_val_1 = train_data_1.drop(["cancer_code","result","index"],axis=1)
test_y_val_1 = test_data_1.result
test_x_val_1 = test_data_1.drop(["cancer_code","result","index"],axis=1)

In [9]:
train_data_2 = data_2.iloc[train_p_ids]
test_data_2 = data_2.iloc[test_p_ids]
y_val_2 = train_data_2.result
x_val_2 = train_data_2.drop(["cancer_code","result","index"],axis=1)
test_y_val_2 = test_data_2.result
test_x_val_2 = test_data_2.drop(["cancer_code","result","index"],axis=1)

In [10]:
train_data_3 = data_3.iloc[train_p_ids]
test_data_3 = data_3.iloc[test_p_ids]
y_val_3 = train_data_3.result
x_val_3 = train_data_3.drop(["cancer_code","result","index"],axis=1)
test_y_val_3 = test_data_3.result
test_x_val_3 = test_data_3.drop(["cancer_code","result","index"],axis=1)

## Modeling Seperate Model

###  Building seperate model for ensemble(model 1, 2, 3)

In [11]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from keras.models import load_model
Drop_out = 0.5

In [12]:
input_m_1 = Input(shape=(features_1,))
adam = optimizers.Adam()

input_drop_m_1 = Dropout(Drop_out)(input_m_1)
h1_m_1 = Dense(500,activation='relu')(input_drop_m_1)
h1_m_1 = Dropout(Drop_out)(h1_m_1)
h2_m_1 = Dense(500,activation='relu')(h1_m_1)
h2_m_1 = Dropout(Drop_out)(h2_m_1)
h3_m_1 = Dense(200,activation='relu')(h2_m_1) 
h3_m_1 = Dropout(Drop_out)(h3_m_1)
h4_m_1 = Dense(300,activation='relu')(h3_m_1) 
h4_m_1 = Dropout(Drop_out)(h4_m_1)
h5_m_1 = Dense(200,activation='relu')(h4_m_1) 
h5_m_1 = Dropout(Drop_out)(h5_m_1)
h6_m_1 = Dense(100,activation='relu')(h5_m_1) 
h6_m_1 = Dropout(Drop_out)(h6_m_1)
output_m_1 = Dense(1,activation="sigmoid")(h6_m_1) 
model_1 = Model(inputs=input_m_1,outputs=output_m_1)

model_1.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['accuracy'])


model_1.fit(x_val_1, y_val_1, epochs=50, callbacks=[early_stopping], validation_split=0.1, batch_size=50)

model_1.save("../models/DNN/"+str(num)+"_DNN_model_1.h5")
print("model_1 saved model to disk")
print("[1] file_name: ", types[select[0]], "\nsample : {}  \nfeatures : {}".format(sample_1,features_1))

Train on 11810 samples, validate on 1313 samples
Epoch 1/50
11810/11810 [==============================] - 9s 732us/step - loss: 0.8098 - acc: 0.5324 - val_loss: 0.7128 - val_acc: 7.6161e-04
Epoch 2/50
11810/11810 [==============================] - 2s 203us/step - loss: 0.4232 - acc: 0.8008 - val_loss: 0.1956 - val_acc: 0.9474
Epoch 3/50
11810/11810 [==============================] - 3s 222us/step - loss: 0.2652 - acc: 0.8955 - val_loss: 0.0163 - val_acc: 0.9954
Epoch 4/50
11810/11810 [==============================] - 3s 216us/step - loss: 0.2265 - acc: 0.9130 - val_loss: 0.0990 - val_acc: 0.9619
Epoch 5/50
11810/11810 [==============================] - 2s 211us/step - loss: 0.2017 - acc: 0.9222 - val_loss: 0.0360 - val_acc: 0.9901
Epoch 6/50
11810/11810 [==============================] - 3s 219us/step - loss: 0.1942 - acc: 0.9257 - val_loss: 0.0395 - val_acc: 0.9886
Epoch 7/50
11810/11810 [==============================] - 3s 217us/step - loss: 0.1882 - acc: 0.9287 - val_loss: 0.0244

KeyboardInterrupt: 

In [ ]:
'''
model_1_ = load_model("DNN_model_1.h5")
model_1[0]
m1_acc = model_1_.evaluate(test_x_val_1, test_y_val_1)
print(m1_acc)
'''

In [ ]:
input_m_2 = Input(shape=(features_2,))
adam = optimizers.Adam()

input_drop_m_2 = Dropout(Drop_out)(input_m_2)
h1_m_2 = Dense(500,activation='relu')(input_drop_m_2)
h1_m_2 = Dropout(Drop_out)(h1_m_2)
h2_m_2 = Dense(500,activation='relu')(h1_m_2)
h2_m_2 = Dropout(Drop_out)(h2_m_2)
h3_m_2 = Dense(200,activation='relu')(h2_m_2) 
h3_m_2 = Dropout(Drop_out)(h3_m_2)
h4_m_2 = Dense(300,activation='relu')(h3_m_2) 
h4_m_2 = Dropout(Drop_out)(h4_m_2)
h5_m_2 = Dense(200,activation='relu')(h4_m_2) 
h5_m_2 = Dropout(Drop_out)(h5_m_2)
h6_m_2 = Dense(100,activation='relu')(h5_m_2) 
h6_m_2 = Dropout(Drop_out)(h6_m_2)
output_m_2 = Dense(1,activation="sigmoid")(h6_m_2) 
model_2 = Model(inputs=input_m_2,outputs=output_m_2)

model_2.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_2.fit(x_val_2, y_val_2, epochs=50, callbacks=[early_stopping], validation_split=0.1, batch_size=50)

model_2.save("../models/DNN/"+str(num)+"_DNN_model_2.h5")
print("model_2 saved model to disk")
print("[2] file_name: ", types[select[1]], "\nsample : {}  \nfeatures : {}".format(sample_2,features_2))

In [ ]:
input_m_3 = Input(shape=(features_3,))
adam = optimizers.Adam()

input_drop_m_3 = Dropout(Drop_out)(input_m_3)
h1_m_3 = Dense(500,activation='relu')(input_drop_m_3)
h1_m_3 = Dropout(Drop_out)(h1_m_3)
h2_m_3 = Dense(500,activation='relu')(h1_m_3)
h2_m_3 = Dropout(Drop_out)(h2_m_3)
h3_m_3 = Dense(200,activation='relu')(h2_m_3) 
h3_m_3 = Dropout(Drop_out)(h3_m_3)
h4_m_3 = Dense(300,activation='relu')(h3_m_3) 
h4_m_3 = Dropout(Drop_out)(h4_m_3)
h5_m_3 = Dense(200,activation='relu')(h4_m_3) 
h5_m_3 = Dropout(Drop_out)(h5_m_3)
h6_m_3 = Dense(100,activation='relu')(h5_m_3) 
h6_m_3 = Dropout(Drop_out)(h6_m_3)
output_m_3 = Dense(1,activation="sigmoid")(h6_m_3) 
model_3 = Model(inputs=input_m_3,outputs=output_m_3)

model_3.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_3.fit(x_val_3, y_val_3, epochs=50, callbacks=[early_stopping], validation_split=0.1, batch_size=50)

model_3.save("../models/DNN/"+str(num)+"_DNN_model_3.h5")
print("model_3 saved model to disk")
print("[1] file_name: ", types[select[2]], "\nsample : {}  \nfeatures : {}".format(sample_3,features_3))

### Evaluating seperate model's performance

In [ ]:
m_1_tr_loss,m_1_tr_accuracy=model_1.evaluate(x_val_1,y_val_1)
m_2_tr_loss,m_2_tr_accuracy=model_2.evaluate(x_val_2,y_val_2)
m_3_tr_loss,m_3_tr_accuracy=model_3.evaluate(x_val_3,y_val_3)

m_1_loss,m_1_accuracy= model_1.evaluate(test_x_val_1,test_y_val_1)
m_2_loss,m_2_accuracy= model_2.evaluate(test_x_val_2,test_y_val_2)
m_3_loss,m_3_accuracy= model_3.evaluate(test_x_val_3,test_y_val_3)

m_1_predictions = model_1.predict(x_val_1)
labeled_m_1_predictions = np.where(m_1_predictions > 0.5, 1, 0).flatten()
m_1_tr_sensitivity, m_1_tr_specificity = check_correct(labeled_m_1_predictions, y_val_1)
m_1_test_predictions = model_1.predict(test_x_val_1)
labeled_m_1_test_predictions = np.where(m_1_test_predictions > 0.5, 1, 0).flatten()
m_1_sensitivity, m_1_specificity = check_correct(labeled_m_1_test_predictions, test_y_val_1)

m_2_predictions = model_2.predict(x_val_2)
labeled_m_2_predictions = np.where(m_2_predictions > 0.5, 1, 0).flatten()
m_2_tr_sensitivity, m_2_tr_specificity = check_correct(labeled_m_2_predictions, y_val_2)
m_2_test_predictions = model_2.predict(test_x_val_2)
labeled_m_2_test_predictions = np.where(m_2_test_predictions > 0.5, 1, 0).flatten()
m_2_sensitivity, m_2_specificity = check_correct(labeled_m_2_test_predictions, test_y_val_2)

m_3_predictions = model_3.predict(x_val_3)
labeled_m_3_predictions = np.where(m_3_predictions > 0.5, 1, 0).flatten()
m_3_tr_sensitivity, m_3_tr_specificity = check_correct(labeled_m_3_predictions, y_val_3)
m_3_test_predictions = model_3.predict(test_x_val_3)
labeled_m_3_test_predictions = np.where(m_3_test_predictions > 0.5, 1, 0).flatten()
m_3_sensitivity, m_3_specificity = check_correct(labeled_m_3_test_predictions, test_y_val_3)


print(types[select[0]],types[select[1]],types[select[2]])
print("../models/DNN/"+str(m_1_tr_accuracy)+", "+str(m_1_accuracy))
print("\nTrain Accuracy for model 1, 2, 3 : {},{},{}".format(m_1_tr_accuracy,m_2_tr_accuracy,m_3_tr_accuracy))
print("\nTest Accuracy for model 1, 2, 3 : {},{},{}".format(m_1_accuracy,m_2_accuracy,m_3_accuracy))

## Modeling Ensemble model

### Building original ensemble model

In [ ]:
m_1_predictions = model_1.predict(x_val_1)
m_2_predictions = model_2.predict(x_val_2)
m_3_predictions = model_3.predict(x_val_3)
adam = optimizers.Adam()

ensemble_x_val = np.concatenate([m_1_predictions, m_2_predictions, m_3_predictions], axis=1)

ensemble_input = Input(shape=(3,))
h1_ensemble = Dense(2,activation="relu")(ensemble_input)
ensemble_output = Dense(1,activation='sigmoid')(h1_ensemble)
ensemble_model = Model(inputs=ensemble_input, outputs=ensemble_output)

ensemble_model.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

ensemble_model.fit(ensemble_x_val, y_val_1, epochs=10, batch_size = 1, validation_split = 0.1, callbacks=[early_stopping])

ensemble_model.save("../models/DNN/"+str(num)+"_ensemble_DNN.h5")

### Evaluating _DNN Combiner_ ensemble model

In [ ]:
m_1_tr_predictions = model_1.predict(x_val_1)
m_2_tr_predictions = model_2.predict(x_val_2)
m_3_tr_predictions = model_3.predict(x_val_3)

m_1_test_predictions = model_1.predict(test_x_val_1)
m_2_test_predictions = model_2.predict(test_x_val_2)
m_3_test_predictions = model_3.predict(test_x_val_3)

ensemble_x_val = np.concatenate([m_1_tr_predictions, m_2_tr_predictions, m_3_tr_predictions], axis=1)
ensemble_test_x_val = np.concatenate([m_1_test_predictions, m_2_test_predictions, m_3_test_predictions], axis=1)

em_loss,em_accuracy= ensemble_model.evaluate(ensemble_test_x_val,test_y_val_1)

ensemble_predictions = ensemble_model.predict(ensemble_x_val)
labeled_ensemble_predictions = np.where(ensemble_predictions > 0.5, 1, 0).flatten()
em_tr_sensitivity, em_tr_specificity = check_correct(labeled_ensemble_predictions, y_val_1)
ensemble_test_predictions = ensemble_model.predict(ensemble_test_x_val)
labeled_ensemble_test_predictions = np.where(ensemble_test_predictions > 0.5, 1, 0).flatten()
em_sensitivity, em_specificity = check_correct(labeled_ensemble_test_predictions, test_y_val_1)

print("Accuracy for DNN combiner ensemble : {}".format(em_accuracy))

### Evaluating _mean_ ensemble model

In [ ]:
mean_tr_predictions=(m_1_tr_predictions+m_2_tr_predictions+m_3_tr_predictions)/3
mean_predictions=(m_1_test_predictions+m_2_test_predictions+m_3_test_predictions)/3
labled_predictions = np.where(mean_predictions > 0.5, 1, 0).flatten()
mean_em_accuracy = sum(labled_predictions==test_y_val_1.values)/len(test_y_val_1)

labeled_mean_em_predictions = np.where(mean_tr_predictions > 0.5, 1, 0).flatten()
mean_em_tr_sensitivity, mean_em_tr_specificity = check_correct(labeled_mean_em_predictions, y_val_1)
labeled_mean_em_test_predictions = np.where(mean_predictions > 0.5, 1, 0).flatten()
mean_em_sensitivity, mean_em_specificity = check_correct(labeled_mean_em_test_predictions, test_y_val_1)
mean_em_tr_accuracy = sum(labeled_mean_em_predictions==y_val_1.values)/len(y_val_1)

print("Accuracy for mean ensemble : {}".format(mean_em_accuracy))

## Transferred Ensemble Modeling 

### Making new input data for t-ensemble

In [ ]:
from keras.models import Model
model = Model(inputs=[input_m_1], outputs=[h6_m_1])
results_m_1 = model.predict([x_val_1])

model = Model(inputs=[input_m_2], outputs=[h6_m_2])
results_m_2 = model.predict([x_val_2])

model = Model(inputs=[input_m_3], outputs=[h6_m_3])
results_m_3 = model.predict([x_val_3])

t_ensemble_x_val = np.concatenate([results_m_1, results_m_2, results_m_3], axis=1)
print(t_ensemble_x_val.shape)

### Modeling t-ensemble  

In [ ]:
t_ensemble_input = Input(shape=(t_ensemble_x_val.shape[1],))
t_ensemble_h1 = Dense(300,activation='relu')(t_ensemble_input)
t_ensemble_h1 = Dropout(Drop_out)(t_ensemble_h1)
t_ensemble_h2 = Dense(150,activation='relu')(t_ensemble_h1)
t_ensemble_h2 = Dropout(Drop_out)(t_ensemble_h2)
t_ensemble_h3 = Dense(100,activation='relu')(t_ensemble_h2)
t_ensemble_h3 = Dropout(Drop_out)(t_ensemble_h3)
t_ensemble_h4 = Dense(50,activation='relu')(t_ensemble_h3)
t_ensemble_h4 = Dropout(Drop_out)(t_ensemble_h4)
t_ensemble_output = Dense(1,activation='sigmoid')(t_ensemble_h4)
adam = optimizers.Adam()

t_ensemble_model = Model(inputs=[t_ensemble_input],outputs=[t_ensemble_output])
t_ensemble_model.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

t_ensemble_model.fit(t_ensemble_x_val, y_val_1, epochs=100, batch_size=20, validation_split = 0.1, callbacks=[early_stopping])

ensemble_model.save("../models/DNN/"+str(num)+"_t_ensemble_DNN.h5")

### Evaluating t-ensemble

In [ ]:
model = Model(inputs=[input_m_1], outputs=[h6_m_1])
test_results_m_1 = model.predict([test_x_val_1])
model = Model(inputs=[input_m_2], outputs=[h6_m_2])
test_results_m_2 = model.predict([test_x_val_2])
model = Model(inputs=[input_m_3], outputs=[h6_m_3])
test_results_m_3 = model.predict([test_x_val_3])

t_ensemble_test_x_val = np.concatenate([test_results_m_1, test_results_m_2, test_results_m_3], axis=1)
t_em_accuracy = t_ensemble_model.evaluate(t_ensemble_test_x_val,test_y_val_1)[1]
t_em_tr_accuracy = t_ensemble_model.evaluate(t_ensemble_x_val,y_val_1)[1]

t_em_predictions = t_ensemble_model.predict(t_ensemble_x_val)
labeled_t_em_predictions = np.where(t_em_predictions > 0.5, 1, 0).flatten()
t_em_tr_sensitivity, t_em_tr_specificity = check_correct(labeled_t_em_predictions, y_val_1)
t_em_test_predictions = t_ensemble_model.predict(t_ensemble_test_x_val)
labeled_t_em_test_predictions = np.where(t_em_test_predictions > 0.5, 1, 0).flatten()
t_em_sensitivity, t_em_specificity = check_correct(labeled_t_em_test_predictions, test_y_val_1)


print("\nAccuracy for t-ensemble: ",t_em_accuracy )

## Performance Comparison

In [ ]:
label = ["model_1","model_2","model_3","mean-em","d-comb em","t-em"]
accuracy = [m_1_accuracy,m_2_accuracy,m_3_accuracy,mean_em_accuracy,em_accuracy,t_em_accuracy ]
print("model_1: "+str(accuracy[0])+"\nmodel_2: "+str(accuracy[1])+"\nmodel_3: "+str(accuracy[2])+"\nmean-em: "+str(accuracy[3])+"\nd-comb em: "+str(accuracy[4])+"\nt-em: "+str(accuracy[5]))

In [ ]:
def plot_bar_x():
    # this is for plotting purpose
    plt.figure(figsize=(30,20))
    axes = plt.gca()
    axes.set_ylim([min(m_1_accuracy,m_2_accuracy,m_3_accuracy,mean_em_accuracy,em_accuracy,t_em_accuracy)-0.02,1])
    index = np.arange(len(label))
    plt.bar(index, accuracy,color=['red', 'orange', 'yellow', "green",'blue', 'purple',],alpha=0.5,width=0.3)
    plt.xlabel('Method', fontsize=35)
    plt.ylabel('Accuracy', fontsize=35)
    plt.yticks(fontsize=30)    
    plt.xticks(index, label, fontsize=30, rotation=90)
    plt.title('Performance Comparison for each Ensemble Model',fontsize=40)
    plt.show()

In [ ]:
plot_bar_x()

In [ ]:
model_box = [types[select[0]], types[select[1]], types[select[2]], "mean_em","DNN_em","t_em"]
tr_accuracy_box = [m_1_tr_accuracy,m_2_tr_accuracy,m_3_tr_accuracy,mean_em_tr_accuracy,em_accuracy,t_em_tr_accuracy]
ts_accuracy_box = [m_1_accuracy,m_2_accuracy,m_3_accuracy,mean_em_accuracy,em_accuracy,t_em_accuracy]
tr_sensitivity_box = [m_1_tr_sensitivity,m_2_tr_sensitivity,m_3_tr_sensitivity,mean_em_tr_sensitivity,em_sensitivity,t_em_tr_sensitivity]
ts_sensitivity_box = [m_1_sensitivity,m_2_sensitivity,m_3_sensitivity,mean_em_sensitivity,em_sensitivity,t_em_sensitivity]
tr_specificity_box = [m_1_tr_specificity,m_2_tr_specificity,m_3_tr_specificity,mean_em_tr_specificity,em_specificity,t_em_tr_specificity]
ts_specificity_box = [m_1_specificity,m_2_specificity,m_3_specificity,mean_em_specificity,em_specificity,t_em_specificity]
df = pd.DataFrame(data={"model":model_box, "tr_accuracy":tr_accuracy_box, "tr_sensitivity":tr_sensitivity_box, "tr_specificity":tr_specificity_box,"ts_accuracy":ts_accuracy_box, "ts_sensitivity":ts_sensitivity_box, "ts_specificity":ts_specificity_box})
df.to_csv("../result/"+str(num)+"_result.csv", index=False, header=True, columns= ["model", "tr_accuracy",  "tr_sensitivity", "tr_specificity","ts_accuracy",  "ts_sensitivity", "ts_specificity"])

In [ ]:
num=num+1
print(num)